In [1]:
import os, json, random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device


device(type='mps')

In [2]:
csv_path = "bank-additional-full-processed.csv"
assert os.path.exists(csv_path), f"找不到文件：{csv_path}"

df = pd.read_csv(csv_path, sep=None, engine="python")
print("Data shape:", df.shape)
df.head()


Data shape: (41188, 21)


age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  
0          93.994          -36.4      4.857       5191.0  no  
1          93.994          -36.4      4.857       5191.0  no  
2          93.994          -36.4      4.857       5191.0  no  
3          93.994          -36.4      4.857       5191.0  no  
4          93.994          -36.4      4.857       5191.0  no  

[5 rows x 21 columns]

In [3]:
df = df.copy().dropna(axis=1, how="all")
target_col = "y" if "y" in df.columns else df.columns[-1]
print("Target column:", target_col)

y_raw = df[target_col]
X_df = df.drop(columns=[target_col])

def to_binary_labels(s: pd.Series) -> np.ndarray:
    if s.dtype == "O" or str(s.dtype).startswith("category"):
        s2 = s.astype(str).str.strip().str.lower()
        if set(s2.unique()) <= {"yes", "no"}:
            return (s2 == "yes").astype(np.int64).values
        if set(s2.unique()) <= {"1", "0"}:
            return (s2 == "1").astype(np.int64).values
    if pd.api.types.is_bool_dtype(s):
        return s.astype(np.int64).values
    if pd.api.types.is_numeric_dtype(s):
        uniq = pd.unique(s.dropna())
        if len(uniq) <= 2:
            uniq_sorted = np.sort(uniq.astype(float))
            pos_val = uniq_sorted[-1]
            return (s.astype(float) == pos_val).astype(np.int64).values
    s2 = s.astype(str).str.strip().str.lower()
    return s2.isin(["yes","1","true","pos","positive"]).astype(np.int64).values

y = to_binary_labels(y_raw)
print("Positive rate:", float(y.mean()))
X_df.head()


Target column: y
Positive rate: 0.11265417111780131


age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  duration  campaign  pdays  previous     poutcome  \
0   may         mon       261         1    999         0  nonexistent   
1   may         mon       149         1    999         0  nonexistent   
2   may         mon       226         1    999         0  nonexistent   
3   may         mon       151         1    999         0  nonexistent   
4   may         mon       307         1    999         0  nonexistent   

   emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed  
0           1.1          93.994          -36.4      4.857       5191.0  
1           1.1          93.994          -36.4      4.857       5191.0  
2           1.1          93.994          -36.4      4.857       5191.0  
3           1.1          93.994          -36.4      4.857       5191.0  
4           1.1          93.994          -36.4      4.857       5191.0

In [4]:
X_train_df, X_temp_df, y_train, y_temp = train_test_split(
    X_df, y, test_size=0.30, random_state=42, stratify=y
)
X_val_df, X_test_df, y_val, y_test = train_test_split(
    X_temp_df, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Train/Val/Test:", X_train_df.shape, X_val_df.shape, X_test_df.shape)
print("Pos rate:", y_train.mean(), y_val.mean(), y_test.mean())


Train/Val/Test: (28831, 20) (6178, 20) (6179, 20)
Pos rate: 0.11265651555617218 0.11265781806409841 0.11263958569347791


In [5]:
cat_cols = [c for c in X_train_df.columns if X_train_df[c].dtype == "object" or str(X_train_df[c].dtype).startswith("category")]
num_cols = [c for c in X_train_df.columns if c not in cat_cols]
print(f"#cat={len(cat_cols)}, #num={len(num_cols)}")
print("Example cat cols:", cat_cols[:10])

# 为每个类别列建立 vocab：0 作为 <unk>
cat_vocabs = {}
cat_sizes = {}
for c in cat_cols:
    vals = X_train_df[c].astype(str).fillna("").str.strip()
    uniq = sorted(vals.unique().tolist())
    vocab = {v: (i+1) for i, v in enumerate(uniq)}  # 1..K
    cat_vocabs[c] = vocab
    cat_sizes[c] = len(vocab) + 1  # +unk(0)

cat_sizes


#cat=10, #num=10
Example cat cols: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


{'job': 13,
 'marital': 5,
 'education': 9,
 'default': 4,
 'housing': 4,
 'loan': 4,
 'contact': 3,
 'month': 11,
 'day_of_week': 6,
 'poutcome': 4}

In [6]:
def encode_cats(df_part: pd.DataFrame, cat_cols, cat_vocabs):
    Xc = np.zeros((len(df_part), len(cat_cols)), dtype=np.int64)
    for j, c in enumerate(cat_cols):
        vocab = cat_vocabs[c]
        vals = df_part[c].astype(str).fillna("").str.strip().values
        Xc[:, j] = np.array([vocab.get(v, 0) for v in vals], dtype=np.int64)
    return Xc

scaler = StandardScaler()
Xn_train = scaler.fit_transform(X_train_df[num_cols].values.astype(np.float32)) if len(num_cols) > 0 else np.zeros((len(X_train_df), 0), np.float32)
Xn_val   = scaler.transform(X_val_df[num_cols].values.astype(np.float32)) if len(num_cols) > 0 else np.zeros((len(X_val_df), 0), np.float32)
Xn_test  = scaler.transform(X_test_df[num_cols].values.astype(np.float32)) if len(num_cols) > 0 else np.zeros((len(X_test_df), 0), np.float32)

Xc_train = encode_cats(X_train_df, cat_cols, cat_vocabs) if len(cat_cols) > 0 else np.zeros((len(X_train_df), 0), np.int64)
Xc_val   = encode_cats(X_val_df,   cat_cols, cat_vocabs) if len(cat_cols) > 0 else np.zeros((len(X_val_df), 0), np.int64)
Xc_test  = encode_cats(X_test_df,  cat_cols, cat_vocabs) if len(cat_cols) > 0 else np.zeros((len(X_test_df), 0), np.int64)

print("Cats:", Xc_train.shape, "Nums:", Xn_train.shape)


Cats: (28831, 10) Nums: (28831, 10)


In [7]:
class TabDataset(Dataset):
    def __init__(self, Xc, Xn, y):
        self.Xc = torch.from_numpy(Xc)  # int64
        self.Xn = torch.from_numpy(Xn.astype(np.float32))  # float32
        self.y  = torch.from_numpy(y.astype(np.float32)).view(-1, 1)  # float32

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return self.Xc[idx], self.Xn[idx], self.y[idx]

batch_size = 512
train_loader = DataLoader(TabDataset(Xc_train, Xn_train, y_train), batch_size=batch_size, shuffle=True, num_workers=0)
val_loader   = DataLoader(TabDataset(Xc_val,   Xn_val,   y_val),   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(TabDataset(Xc_test,  Xn_test,  y_test),  batch_size=batch_size, shuffle=False, num_workers=0)

n_cat = Xc_train.shape[1]
n_num = Xn_train.shape[1]
n_tokens = n_cat + n_num
n_cat, n_num, n_tokens


(10, 10, 20)

In [8]:
class TabTransformer(nn.Module):
    def __init__(
        self,
        cat_sizes: list,           # 每个类别列的词表大小
        n_num: int,
        d_model: int = 64,
        n_heads: int = 8,
        n_layers: int = 4,
        ff_mult: int = 4,
        dropout: float = 0.15,
    ):
        super().__init__()
        self.n_cat = len(cat_sizes)
        self.n_num = n_num
        self.d_model = d_model

        # categorical embeddings
        self.cat_embeds = nn.ModuleList([
            nn.Embedding(sz, d_model) for sz in cat_sizes
        ])

        # numeric -> token embedding: x * W + b （每个数值特征一组参数）
        if n_num > 0:
            self.num_W = nn.Parameter(torch.randn(n_num, d_model) * 0.02)
            self.num_b = nn.Parameter(torch.zeros(n_num, d_model))
        else:
            self.num_W = None
            self.num_b = None

        # feature (token position) embedding
        self.feat_emb = nn.Parameter(torch.randn(self.n_cat + self.n_num, d_model) * 0.02)

        # CLS token
        self.cls = nn.Parameter(torch.randn(1, 1, d_model) * 0.02)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * ff_mult,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
            norm_first=True,  # 更稳
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1)  # logits
        )

    def forward(self, Xc, Xn):
        # Xc: [B, n_cat] int64
        # Xn: [B, n_num] float32
        B = Xc.shape[0]

        tokens = []

        # categorical tokens
        if self.n_cat > 0:
            for j, emb in enumerate(self.cat_embeds):
                tokens.append(emb(Xc[:, j]))  # [B, d_model]

        # numeric tokens
        if self.n_num > 0:
            # [B, n_num, 1] * [n_num, d_model] -> [B, n_num, d_model]
            xn = Xn.unsqueeze(-1)
            num_tok = xn * self.num_W.unsqueeze(0) + self.num_b.unsqueeze(0)
            tokens.append(num_tok)  # [B, n_num, d_model]

        # stack tokens
        if len(tokens) == 0:
            raise RuntimeError("No features found (n_cat=0 and n_num=0).")

        if self.n_cat > 0 and self.n_num > 0:
            cat_tok = torch.stack(tokens[:self.n_cat], dim=1)          # [B, n_cat, d_model]
            num_tok = tokens[-1]                                       # [B, n_num, d_model]
            x = torch.cat([cat_tok, num_tok], dim=1)                   # [B, n_tokens, d_model]
        elif self.n_cat > 0:
            x = torch.stack(tokens, dim=1)                             # [B, n_cat, d_model]
        else:
            x = tokens[0]                                              # [B, n_num, d_model]

        # add feature embeddings
        x = x + self.feat_emb.unsqueeze(0)

        # prepend CLS
        cls = self.cls.expand(B, 1, self.d_model)
        x = torch.cat([cls, x], dim=1)                                 # [B, 1+n_tokens, d_model]

        x = self.encoder(x)
        cls_out = x[:, 0, :]
        logits = self.head(cls_out)
        return logits

cat_sizes_list = [cat_sizes[c] for c in cat_cols]
model = TabTransformer(
    cat_sizes=cat_sizes_list,
    n_num=n_num,
    d_model=64,
    n_heads=8,
    n_layers=4,
    ff_mult=4,
    dropout=0.15
).to(device)

model


/opt/anaconda3/envs/cs231n/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


TabTransformer(
  (cat_embeds): ModuleList(
    (0): Embedding(13, 64)
    (1): Embedding(5, 64)
    (2): Embedding(9, 64)
    (3-5): 3 x Embedding(4, 64)
    (6): Embedding(3, 64)
    (7): Embedding(11, 64)
    (8): Embedding(6, 64)
    (9): Embedding(4, 64)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.15, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.15, inplace=False)
        (dropout2): Dropout(p=0.15, inplace=False)
      )
    )
  )
  (head): Sequential(
    (0): Lay

In [9]:
n_pos = int(y_train.sum())
n_neg = int(len(y_train) - n_pos)
ratio = n_neg / max(n_pos, 1)

pos_weight = torch.tensor([ratio**0.5], dtype=torch.float32, device=device)  # 温和偏向 recall
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2)

float(pos_weight.item())


2.8065173625946045

In [10]:
@torch.no_grad()
def predict_probs(model, loader):
    model.eval()
    probs_list, y_list = [], []
    for Xc, Xn, yb in loader:
        Xc = Xc.to(device)
        Xn = Xn.to(device)
        logits = model(Xc, Xn)
        probs = torch.sigmoid(logits).detach().cpu().numpy().ravel()
        probs_list.append(probs)
        y_list.append(yb.numpy().ravel())
    probs = np.concatenate(probs_list)
    y_true = np.concatenate(y_list).astype(int)
    return y_true, probs

@torch.no_grad()
def eval_auc(model, loader):
    y_true, probs = predict_probs(model, loader)
    return roc_auc_score(y_true, probs) if len(np.unique(y_true)) == 2 else np.nan


In [11]:
from copy import deepcopy

max_epochs = 100
patience = 12

best_state = None
best_val_auc = -1.0
bad_epochs = 0

for epoch in range(1, max_epochs + 1):
    model.train()
    total_loss = 0.0
    n = 0

    for Xc, Xn, yb in train_loader:
        Xc = Xc.to(device)
        Xn = Xn.to(device)
        yb = yb.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits = model(Xc, Xn)
        loss = criterion(logits, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()

        total_loss += loss.item() * yb.size(0)
        n += yb.size(0)

    train_loss = total_loss / max(n, 1)
    val_auc = eval_auc(model, val_loader)
    scheduler.step(val_auc)

    lr_now = optimizer.param_groups[0]["lr"]
    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_auc={val_auc:.4f} | lr={lr_now:.2e}")

    if val_auc > best_val_auc:
        best_val_auc = val_auc
        best_state = deepcopy(model.state_dict())
        bad_epochs = 0
    else:
        bad_epochs += 1
        if bad_epochs >= patience:
            print(f"Early stopping. Best val_auc={best_val_auc:.4f}")
            break

if best_state is not None:
    model.load_state_dict(best_state)


Epoch 01 | train_loss=0.4754 | val_auc=0.9359 | lr=5.00e-04
Epoch 02 | train_loss=0.3560 | val_auc=0.9399 | lr=5.00e-04
Epoch 03 | train_loss=0.3386 | val_auc=0.9439 | lr=5.00e-04
Epoch 04 | train_loss=0.3380 | val_auc=0.9436 | lr=5.00e-04
Epoch 05 | train_loss=0.3331 | val_auc=0.9453 | lr=5.00e-04
Epoch 06 | train_loss=0.3251 | val_auc=0.9457 | lr=5.00e-04
Epoch 07 | train_loss=0.3211 | val_auc=0.9472 | lr=5.00e-04
Epoch 08 | train_loss=0.3172 | val_auc=0.9461 | lr=5.00e-04
Epoch 09 | train_loss=0.3153 | val_auc=0.9484 | lr=5.00e-04
Epoch 10 | train_loss=0.3114 | val_auc=0.9486 | lr=5.00e-04
Epoch 11 | train_loss=0.3116 | val_auc=0.9482 | lr=5.00e-04
Epoch 12 | train_loss=0.3078 | val_auc=0.9485 | lr=5.00e-04
Epoch 13 | train_loss=0.3091 | val_auc=0.9481 | lr=2.50e-04
Epoch 14 | train_loss=0.3032 | val_auc=0.9497 | lr=2.50e-04
Epoch 15 | train_loss=0.3040 | val_auc=0.9494 | lr=2.50e-04
Epoch 16 | train_loss=0.3003 | val_auc=0.9498 | lr=2.50e-04
Epoch 17 | train_loss=0.2973 | val_auc=0

In [18]:
def pick_threshold_max_acc_with_recall(y_true, probs, r_min=0.85, grid=None):
    if grid is None:
        grid = np.linspace(0.01, 0.99, 99)

    best = None
    for t in grid:
        pred = (probs >= t).astype(int)
        acc = accuracy_score(y_true, pred)
        rec = recall_score(y_true, pred, zero_division=0)
        if rec >= r_min:
            if best is None or acc > best["acc"] or (acc == best["acc"] and rec > best["rec"]):
                best = {"t": float(t), "acc": float(acc), "rec": float(rec)}

    if best is None:
        print("Warning: 未找到满足 recall 下限的阈值，请降低 r_min 或调整训练。")
        # 退化：优先最大化 recall，其次 acc
        best2 = None
        for t in grid:
            pred = (probs >= t).astype(int)
            acc = accuracy_score(y_true, pred)
            rec = recall_score(y_true, pred, zero_division=0)
            if best2 is None or rec > best2["rec"] or (rec == best2["rec"] and acc > best2["acc"]):
                best2 = {"t": float(t), "acc": float(acc), "rec": float(rec)}
        best = best2

    return best["t"], best["acc"], best["rec"]

val_y_true, val_probs = predict_probs(model, val_loader)

r_min = 0.75  # 你可以改成 0.84 / 0.90 等
best_t, val_acc, val_rec = pick_threshold_max_acc_with_recall(val_y_true, val_probs, r_min=r_min)
print("Chosen threshold:", best_t, "| val_acc:", val_acc, "| val_recall:", val_rec)


Chosen threshold: 0.61 | val_acc: 0.9135642602784072 | val_recall: 0.7586206896551724


In [19]:
test_y_true, test_probs = predict_probs(model, test_loader)
test_pred = (test_probs >= best_t).astype(int)

print(f"Using threshold: {best_t:.2f}")
print(f"Test AUC: {roc_auc_score(test_y_true, test_probs):.4f}")
print(f"Test ACC: {accuracy_score(test_y_true, test_pred):.4f}")
print(f"Test Precision: {precision_score(test_y_true, test_pred, zero_division=0):.4f}")
print(f"Test Recall: {recall_score(test_y_true, test_pred, zero_division=0):.4f}")
print(f"Test F1: {f1_score(test_y_true, test_pred, zero_division=0):.4f}")

print("\nConfusion Matrix:\n", confusion_matrix(test_y_true, test_pred))
print("\nClassification Report:\n", classification_report(test_y_true, test_pred, digits=4, zero_division=0))


Using threshold: 0.61
Test AUC: 0.9477
Test ACC: 0.9081
Test Precision: 0.5688
Test Recall: 0.7601
Test F1: 0.6507

Confusion Matrix:
 [[5082  401]
 [ 167  529]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9682    0.9269    0.9471      5483
           1     0.5688    0.7601    0.6507       696

    accuracy                         0.9081      6179
   macro avg     0.7685    0.8435    0.7989      6179
weighted avg     0.9232    0.9081    0.9137      6179



In [20]:
import joblib

save_dir = "bank_tabtransformer_artifacts"
os.makedirs(save_dir, exist_ok=True)

torch.save(model.state_dict(), os.path.join(save_dir, "tabtransformer_state_dict.pt"))
joblib.dump(scaler, os.path.join(save_dir, "num_scaler.joblib"))
joblib.dump(cat_vocabs, os.path.join(save_dir, "cat_vocabs.joblib"))

meta = {
    "model_name": "TabTransformer",
    "cat_cols": cat_cols,
    "num_cols": num_cols,
    "cat_sizes": {c: int(cat_sizes[c]) for c in cat_cols},
    "n_num": int(n_num),
    "d_model": 64,
    "n_heads": 8,
    "n_layers": 4,
    "ff_mult": 4,
    "dropout": 0.15,
    "pos_weight": float(pos_weight.item()),
    "threshold": float(best_t),
    "r_min": float(r_min),
    "best_val_auc": float(best_val_auc),
    "target_col": target_col
}
with open(os.path.join(save_dir, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("Saved to:", os.path.abspath(save_dir))
print("Files:", os.listdir(save_dir))


Saved to: /Users/jianxiang/PhD1/课程/统计学习/bank_tabtransformer_artifacts
Files: ['num_scaler.joblib', 'tabtransformer_state_dict.pt', 'cat_vocabs.joblib', 'meta.json']
